# MCP using local servers

In this notebook, we will see
- Chain of Thought prompting for tools
- Running a local http mcp server
  - Connecting to http server as a client
  

Inside the `mcp_servers` folder, we have another mcp tools file `mcp_http_server.py`.

In this python file or mcp server file, we have two mcp tools
1. the `read_local_file` mcp tool - which is same as the one used in notebook [1_intro_to_MCP](1_intro_to_MCP.ipynb)
2. A new mcp tool called `execute_python_code`
   1. We can use this tool to directly run python code

We will combine both tools together.
1. We will read a python script from local machine via the mcp tool `read_local_file`
2. Then we will send the output of the first tool to the second tool which is `execute_python_code`
3. We will us Chain of Thought prompting for this execution

In [ ]:
import ollama
from mcp import ClientSession
from mcp.client.sse import sse_client as http_client

In [ ]:
LOCAL_SERVER_URL = "http://127.0.0.1:8000/sse"

In [ ]:
SYSTEM_PROMPT = (
                "You are a helpful assistant with access to tools. "
                "If a task requires multiple steps (e.g., reading a file then running it), "
                "perform them one by one. Use the output of one tool to inform the next. "
                "If you need to read a file, use the file reading tool first. If you need to execute code, use the code execution tool."
                "After getting result from executing code, give the output as the final answer."
                "When you have the final answer, summarize it for the user."
                "If the python files have syntax error then output the syntax error as the final answer without trying to execute it and tell what the error is"
            )

In [ ]:
async def run_chainable_agent(user_prompt, system_prompt, model):
    # Define the conversation history
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]

    async with http_client(LOCAL_SERVER_URL) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            # 1. Fetch and format tools for Ollama
            tools_result = await session.list_tools()
            ollama_tools = [{
                "type": "function",
                "function": {
                    "name": t.name,
                    "description": t.description,
                    "parameters": t.inputSchema,
                }
            } for t in tools_result.tools]

            print(f"🚀 Starting agent loop for: '{user_prompt}'")
            
            # 2. Start the Loop (Max 5 turns to prevent infinite loops)
            for turn in range(5):
                print(f"\n--- Turn {turn + 1} ---")
                
                response = ollama.chat(
                    model=model,
                    messages=messages,
                    tools=ollama_tools
                )
                
                assistant_msg = response['message']
                messages.append(assistant_msg) # Record the model's reasoning/call

                # Check if the model wants to call tools
                if not assistant_msg.get('tool_calls'):
                    print("✅ Final answer reached.")
                    return assistant_msg['content']

                # 3. Execute Tool Calls
                for tool_call in assistant_msg['tool_calls']:
                    t_name = tool_call['function']['name']
                    t_args = tool_call['function']['arguments']
                    
                    print(f"🛠️ Tool Call: {t_name}({t_args})")
                    
                    # Execute the tool via MCP
                    result = await session.call_tool(t_name, t_args)
                    
                    # Append the result back to the conversation
                    messages.append({
                        'role': 'tool',
                        'content': str(result.content),
                        'name': t_name
                    })
                    print(f"📥 Tool Result received.")

            return "⚠️ Loop limit reached without a final answer."



In [ ]:
query = "Read fibonacci_numbers.py in the scripts folder and then execute it. Path to file=scripts/fibonacci_numbers.py"
MODEL = "llama3.2:3b"

result = await run_chainable_agent(query, SYSTEM_PROMPT, MODEL)
print(f"\nRESULT:\n{result}")

When we run the cell above we get an error because our mcp tool is not active on the server. For this we first have to start the local server. 
To do this:
1. Open terminal
2. Navigate to this repository
3. Activate your virtual environment
4. Run `python mcp_servers/mcp_http_server.py`


After executing this command, you will see that the server has started and now actively listening for connections

#### Check if the server is listening and we can connect to the server
1. Open another terminal and navigate to the repository
2. Activate the virtual environment
3. Run `curl -v http://127.0.0.1:8000/sse`
4. If the connection is established to the server, you should see something like `HTTP/1.1 200 OK` on your terminal
5. On the server side, that is the terminal window where server is running, you will also see `200 OK` if successfully connected
6. Exit the curl command on the client side with `control + c/d`


#### Connect LLM to the MCP server
Re-ru the above cell again, to see if we can connect to the server


## DIY - Do It Yourself

Now try to implement new mcp tools and make them work for you. You can start with very simple math tools.
